In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = []
                list_div = html.xpath('//div[@class="product info detailed"]/div/div')
                for i in range(len(list_div)):
                    if len(list_div[i].xpath('./a/text()')) != 0 and list_div[i].xpath('./a/text()')[0].strip() == 'Cross Reference':
                        list_tr = list_div[i+1].xpath('./table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

                for tr in list_tr:
                    df_temp.loc[0, 'OE_' + tr.xpath('./td[1]/text()')[0].strip()] = tr.xpath('./td[2]/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./oe-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./oe_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2484

[ok] - A2267
[尝试次数：1] - [剩余数量：2464] - [当前时间：21:41:58]

[ok] - A2092
[尝试次数：1] - [剩余数量：2463] - [当前时间：21:41:58]

[ok] - A2041
[尝试次数：1] - [剩余数量：2462] - [当前时间：21:41:59]

[ok] - A2120
[尝试次数：1] - [剩余数量：2461] - [当前时间：21:41:59]

[ok] - A2220
[尝试次数：1] - [剩余数量：2460] - [当前时间：21:41:59]

[ok] - A2247
[尝试次数：1] - [剩余数量：2459] - [当前时间：21:41:59]

[ok] - A2226
[尝试次数：1] - [剩余数量：2458] - [当前时间：21:42:00]

[ok] - A2285
[尝试次数：1] - [剩余数量：2457] - [当前时间：21:42:01]

[ok] - A2091
[尝试次数：1] - [剩余数量：2456] - [当前时间：21:42:01]

[ok] - A2291
[尝试次数：1] - [剩余数量：2455] - [当前时间：21:42:03]

[ok] - A2053
[尝试次数：1] - [剩余数量：2454] - [当前时间：21:42:03]

[ok] - A2296
[尝试次数：1] - [剩余数量：2453] - [当前时间：21:42:03]

[ok] - A2283
[尝试次数：1] - [剩余数量：2452] - [当前时间：21:42:04]

[ok] - A2287
[尝试次数：1] - [剩余数量：2451] - [当前时间：21:42:06]

[ok] - A2298
[尝试次数：1] - [剩余数量：2450] - [当前时间：21:42:06]

[ok] - A2319
[尝试次数：1] - [剩余数量：2449] - [当前时间：21:42:08]

[ok] - A2221
[尝试次数：1] - [剩余数量：2448] - [当前时间：21:42:08]

[ok] - A2280
[尝试次数：2] - [剩余数量：2447] - [当前时间：21:42:09]



[ok] - A2727
[尝试次数：1] - [剩余数量：2315] - [当前时间：21:44:51]

[ok] - A2642
[尝试次数：2] - [剩余数量：2314] - [当前时间：21:44:51]

[ok] - A2726
[尝试次数：1] - [剩余数量：2313] - [当前时间：21:44:52]

[ok] - A2738
[尝试次数：1] - [剩余数量：2312] - [当前时间：21:44:52]

[ok] - A2747
[尝试次数：1] - [剩余数量：2311] - [当前时间：21:44:53]

[ok] - A2731
[尝试次数：1] - [剩余数量：2310] - [当前时间：21:44:53]

[ok] - A2755
[尝试次数：1] - [剩余数量：2309] - [当前时间：21:44:54]

[ok] - A2756
[尝试次数：1] - [剩余数量：2308] - [当前时间：21:44:54]

[ok] - A2763
[尝试次数：1] - [剩余数量：2307] - [当前时间：21:44:54]

[ok] - A2640
[尝试次数：3] - [剩余数量：2306] - [当前时间：21:44:54]

[ok] - A2775
[尝试次数：1] - [剩余数量：2305] - [当前时间：21:44:56]

[ok] - A2785
[尝试次数：1] - [剩余数量：2304] - [当前时间：21:44:58]

[ok] - A2788HY
[尝试次数：1] - [剩余数量：2303] - [当前时间：21:45:00]

[ok] - A2777
[尝试次数：1] - [剩余数量：2302] - [当前时间：21:45:00]

[ok] - A2767
[尝试次数：1] - [剩余数量：2301] - [当前时间：21:45:01]

[ok] - A2791HY
[尝试次数：1] - [剩余数量：2300] - [当前时间：21:45:04]

[ok] - A2661HY
[尝试次数：1] - [剩余数量：2299] - [当前时间：21:45:04]

[ok] - A2465
[尝试次数：2] - [剩余数量：2298] - [当前时间：21:45:07]

[ok]

[ok] - A3031
[尝试次数：1] - [剩余数量：2166] - [当前时间：21:47:27]

[ok] - A2961
[尝试次数：2] - [剩余数量：2165] - [当前时间：21:47:27]

[ok] - A4009
[尝试次数：2] - [剩余数量：2164] - [当前时间：21:47:28]

[ok] - A4019
[尝试次数：1] - [剩余数量：2163] - [当前时间：21:47:29]

[ok] - A4026
[尝试次数：1] - [剩余数量：2162] - [当前时间：21:47:31]

[ok] - A3003
[尝试次数：2] - [剩余数量：2161] - [当前时间：21:47:32]

[ok] - A4008
[尝试次数：2] - [剩余数量：2160] - [当前时间：21:47:32]

[ok] - A4032
[尝试次数：1] - [剩余数量：2159] - [当前时间：21:47:33]

[ok] - A4024
[尝试次数：1] - [剩余数量：2158] - [当前时间：21:47:33]

[ok] - A4045
[尝试次数：1] - [剩余数量：2157] - [当前时间：21:47:34]

[ok] - A4036
[尝试次数：1] - [剩余数量：2156] - [当前时间：21:47:34]

[ok] - A4047
[尝试次数：1] - [剩余数量：2155] - [当前时间：21:47:36]

[ok] - A4030
[尝试次数：1] - [剩余数量：2154] - [当前时间：21:47:36]

[ok] - A4038
[尝试次数：1] - [剩余数量：2153] - [当前时间：21:47:36]

[ok] - A4010
[尝试次数：1] - [剩余数量：2152] - [当前时间：21:47:36]

[ok] - A4051
[尝试次数：1] - [剩余数量：2151] - [当前时间：21:47:38]

[ok] - A4057
[尝试次数：1] - [剩余数量：2150] - [当前时间：21:47:39]

[ok] - A3011
[尝试次数：3] - [剩余数量：2149] - [当前时间：21:47:40]

[ok] - A29

[ok] - A4273
[尝试次数：1] - [剩余数量：2018] - [当前时间：21:49:28]

[ok] - A4309
[尝试次数：1] - [剩余数量：2017] - [当前时间：21:49:28]

[ok] - A4240
[尝试次数：2] - [剩余数量：2016] - [当前时间：21:49:29]

[ok] - A42091
[尝试次数：2] - [剩余数量：2015] - [当前时间：21:49:32]

[ok] - A4310
[尝试次数：1] - [剩余数量：2014] - [当前时间：21:49:36]

[ok] - A4289
[尝试次数：1] - [剩余数量：2013] - [当前时间：21:49:36]

[ok] - A4232
[尝试次数：2] - [剩余数量：2012] - [当前时间：21:49:37]

[ok] - A4281
[尝试次数：1] - [剩余数量：2011] - [当前时间：21:49:39]

[ok] - A4326
[尝试次数：1] - [剩余数量：2010] - [当前时间：21:49:41]

[ok] - A4320
[尝试次数：1] - [剩余数量：2009] - [当前时间：21:49:41]

[ok] - A4311
[尝试次数：1] - [剩余数量：2008] - [当前时间：21:49:44]

[ok] - A4329
[尝试次数：1] - [剩余数量：2007] - [当前时间：21:49:44]

[ok] - A4252
[尝试次数：2] - [剩余数量：2006] - [当前时间：21:49:45]

[ok] - A42079
[尝试次数：2] - [剩余数量：2005] - [当前时间：21:49:46]

[ok] - A4318
[尝试次数：1] - [剩余数量：2004] - [当前时间：21:49:48]

[ok] - A42076
[尝试次数：1] - [剩余数量：2003] - [当前时间：21:49:49]

[ok] - A4345
[尝试次数：1] - [剩余数量：2002] - [当前时间：21:49:51]

[ok] - A4322EL
[尝试次数：1] - [剩余数量：2001] - [当前时间：21:49:54]

[ok] 

[ok] - A4633
[尝试次数：1] - [剩余数量：1869] - [当前时间：21:51:54]

[ok] - A4632
[尝试次数：1] - [剩余数量：1868] - [当前时间：21:51:54]

[ok] - A4616
[尝试次数：1] - [剩余数量：1867] - [当前时间：21:51:55]

[ok] - A4589
[尝试次数：1] - [剩余数量：1866] - [当前时间：21:51:56]

[ok] - A4647
[尝试次数：1] - [剩余数量：1865] - [当前时间：21:51:57]

[ok] - A4526HY
[尝试次数：1] - [剩余数量：1864] - [当前时间：21:51:57]

[ok] - A4640
[尝试次数：1] - [剩余数量：1863] - [当前时间：21:51:59]

[ok] - A4651
[尝试次数：1] - [剩余数量：1862] - [当前时间：21:52:00]

[ok] - A4653
[尝试次数：1] - [剩余数量：1861] - [当前时间：21:52:00]

[ok] - A4701
[尝试次数：1] - [剩余数量：1860] - [当前时间：21:52:02]

[ok] - A4705
[尝试次数：1] - [剩余数量：1859] - [当前时间：21:52:03]

[ok] - A4624
[尝试次数：1] - [剩余数量：1858] - [当前时间：21:52:04]

[ok] - A4694
[尝试次数：1] - [剩余数量：1857] - [当前时间：21:52:06]

[ok] - A4711
[尝试次数：1] - [剩余数量：1856] - [当前时间：21:52:06]

[ok] - A4642
[尝试次数：1] - [剩余数量：1855] - [当前时间：21:52:08]

[ok] - A4903
[尝试次数：1] - [剩余数量：1854] - [当前时间：21:52:09]

[ok] - A4707
[尝试次数：1] - [剩余数量：1853] - [当前时间：21:52:11]

[ok] - A4908
[尝试次数：1] - [剩余数量：1852] - [当前时间：21:52:11]

[ok] - A

...

[ok] - A7349EL
[尝试次数：4] - [剩余数量：674] - [当前时间：22:09:37]

[ok] - AM214
[尝试次数：2] - [剩余数量：673] - [当前时间：22:09:37]

[ok] - AM170
[尝试次数：7] - [剩余数量：672] - [当前时间：22:09:38]

[ok] - AM219IN
[尝试次数：3] - [剩余数量：671] - [当前时间：22:09:39]

[ok] - AM232
[尝试次数：1] - [剩余数量：670] - [当前时间：22:09:39]

[ok] - AM151
[尝试次数：6] - [剩余数量：669] - [当前时间：22:09:40]

[ok] - AM226
[尝试次数：8] - [剩余数量：668] - [当前时间：22:09:41]

[ok] - AM254
[尝试次数：3] - [剩余数量：667] - [当前时间：22:09:42]

[ok] - A7200
[尝试次数：11] - [剩余数量：666] - [当前时间：22:09:43]

[ok] - AM303
[尝试次数：2] - [剩余数量：665] - [当前时间：22:09:44]

[ok] - AM206
[尝试次数：2] - [剩余数量：664] - [当前时间：22:09:44]

[ok] - AM273
[尝试次数：5] - [剩余数量：663] - [当前时间：22:09:45]

[ok] - A7380
[尝试次数：16] - [剩余数量：662] - [当前时间：22:09:46]

[ok] - AM213
[尝试次数：23] - [剩余数量：661] - [当前时间：22:09:47]

[ok] - AM215
[尝试次数：3] - [剩余数量：660] - [当前时间：22:09:48]

[ok] - AM209
[尝试次数：16] - [剩余数量：659] - [当前时间：22:09:48]

[ok] - AM308
[尝试次数：3] - [剩余数量：658] - [当前时间：22:09:49]

[ok] - AM316
[尝试次数：4] - [剩余数量：657] - [当前时间：22:09:50]

[ok] - AM262HY
[尝试次数

[ok] - MK5900
[尝试次数：6] - [剩余数量：523] - [当前时间：22:12:33]

[ok] - A2344
[尝试次数：1] - [剩余数量：522] - [当前时间：22:12:35]

[ok] - A2338
[尝试次数：8] - [剩余数量：521] - [当前时间：22:12:36]

[ok] - A2256
[尝试次数：6] - [剩余数量：520] - [当前时间：22:12:36]

[ok] - A2335
[尝试次数：11] - [剩余数量：519] - [当前时间：22:12:37]

[ok] - A2333
[尝试次数：3] - [剩余数量：518] - [当前时间：22:12:38]

[ok] - A2358
[尝试次数：2] - [剩余数量：517] - [当前时间：22:12:38]

[ok] - A2334
[尝试次数：6] - [剩余数量：516] - [当前时间：22:12:40]

[ok] - A2350
[尝试次数：1] - [剩余数量：515] - [当前时间：22:12:41]

[ok] - A2351
[尝试次数：1] - [剩余数量：514] - [当前时间：22:12:41]

[ok] - A2365
[尝试次数：1] - [剩余数量：513] - [当前时间：22:12:41]

[ok] - A2364
[尝试次数：1] - [剩余数量：512] - [当前时间：22:12:43]

[ok] - A2262
[尝试次数：4] - [剩余数量：511] - [当前时间：22:12:44]

[ok] - A2369
[尝试次数：2] - [剩余数量：510] - [当前时间：22:12:44]

[ok] - A2368
[尝试次数：2] - [剩余数量：509] - [当前时间：22:12:45]

[ok] - A2336
[尝试次数：10] - [剩余数量：508] - [当前时间：22:12:46]

[ok] - MK5511
[尝试次数：4] - [剩余数量：507] - [当前时间：22:12:46]

[ok] - A2384
[尝试次数：1] - [剩余数量：506] - [当前时间：22:12:47]

[ok] - A2375
[尝试次数：5] - 

[ok] - A4401
[尝试次数：1] - [剩余数量：371] - [当前时间：22:15:06]

[ok] - A4436
[尝试次数：1] - [剩余数量：370] - [当前时间：22:15:06]

[ok] - A4366
[尝试次数：1] - [剩余数量：369] - [当前时间：22:15:08]

[ok] - A45012
[尝试次数：1] - [剩余数量：368] - [当前时间：22:15:09]

[ok] - A2998
[尝试次数：2] - [剩余数量：367] - [当前时间：22:15:09]

[ok] - A42013
[尝试次数：1] - [剩余数量：366] - [当前时间：22:15:10]

[ok] - A45006
[尝试次数：1] - [剩余数量：365] - [当前时间：22:15:10]

[ok] - A4368
[尝试次数：2] - [剩余数量：364] - [当前时间：22:15:12]

[ok] - A4574
[尝试次数：1] - [剩余数量：363] - [当前时间：22:15:13]

[ok] - A4514
[尝试次数：1] - [剩余数量：362] - [当前时间：22:15:14]

[ok] - A4050
[尝试次数：1] - [剩余数量：361] - [当前时间：22:15:14]

[ok] - A4563
[尝试次数：1] - [剩余数量：360] - [当前时间：22:15:14]

[ok] - A4618
[尝试次数：1] - [剩余数量：359] - [当前时间：22:15:15]

[ok] - A4258
[尝试次数：3] - [剩余数量：358] - [当前时间：22:15:15]

[ok] - A4266
[尝试次数：2] - [剩余数量：357] - [当前时间：22:15:17]

[ok] - A4628
[尝试次数：1] - [剩余数量：356] - [当前时间：22:15:17]

[ok] - A4444
[尝试次数：2] - [剩余数量：355] - [当前时间：22:15:19]

[ok] - A2913
[尝试次数：1] - [剩余数量：354] - [当前时间：22:15:20]

[ok] - A4644
[尝试次数：1] - [

[ok] - A6279
[尝试次数：1] - [剩余数量：219] - [当前时间：22:17:32]

[ok] - A6355
[尝试次数：1] - [剩余数量：218] - [当前时间：22:17:33]

[ok] - A6382
[尝试次数：1] - [剩余数量：217] - [当前时间：22:17:33]

[ok] - A6225
[尝试次数：1] - [剩余数量：216] - [当前时间：22:17:33]

[ok] - A6417
[尝试次数：1] - [剩余数量：215] - [当前时间：22:17:37]

[ok] - A6331
[尝试次数：1] - [剩余数量：214] - [当前时间：22:17:43]

[ok] - A6363
[尝试次数：1] - [剩余数量：213] - [当前时间：22:17:43]

[ok] - A6247
[尝试次数：2] - [剩余数量：212] - [当前时间：22:17:45]

[ok] - A6459
[尝试次数：1] - [剩余数量：211] - [当前时间：22:17:49]

[ok] - A6407
[尝试次数：1] - [剩余数量：210] - [当前时间：22:17:50]

[ok] - A5430
[尝试次数：4] - [剩余数量：209] - [当前时间：22:17:50]

[ok] - A6255
[尝试次数：1] - [剩余数量：208] - [当前时间：22:17:51]

[ok] - A5723
[尝试次数：1] - [剩余数量：207] - [当前时间：22:17:52]

[ok] - A6253
[尝试次数：3] - [剩余数量：206] - [当前时间：22:17:52]

[ok] - A65001HY
[尝试次数：1] - [剩余数量：205] - [当前时间：22:17:53]

[ok] - A65024
[尝试次数：1] - [剩余数量：204] - [当前时间：22:17:56]

[ok] - A6398
[尝试次数：2] - [剩余数量：203] - [当前时间：22:17:56]

[ok] - A6505
[尝试次数：1] - [剩余数量：202] - [当前时间：22:17:58]

[ok] - A6481
[尝试次数：1] - 

[ok] - A7270
[尝试次数：1] - [剩余数量：67] - [当前时间：22:19:48]

[ok] - A7229
[尝试次数：1] - [剩余数量：66] - [当前时间：22:19:49]

[ok] - A7269
[尝试次数：1] - [剩余数量：65] - [当前时间：22:19:49]

[ok] - A7323HY
[尝试次数：1] - [剩余数量：64] - [当前时间：22:19:51]

[ok] - A7324
[尝试次数：1] - [剩余数量：63] - [当前时间：22:19:52]

[ok] - A7336
[尝试次数：1] - [剩余数量：62] - [当前时间：22:19:54]

[ok] - A7224
[尝试次数：2] - [剩余数量：61] - [当前时间：22:19:54]

[ok] - A7353
[尝试次数：1] - [剩余数量：60] - [当前时间：22:19:55]

[ok] - A7264
[尝试次数：1] - [剩余数量：59] - [当前时间：22:19:56]

[ok] - A6998
[尝试次数：3] - [剩余数量：58] - [当前时间：22:19:56]

[ok] - A72011
[尝试次数：3] - [剩余数量：57] - [当前时间：22:19:56]

[ok] - A7362
[尝试次数：1] - [剩余数量：56] - [当前时间：22:19:57]

[ok] - A7354EL
[尝试次数：1] - [剩余数量：55] - [当前时间：22:19:57]

[ok] - A7364
[尝试次数：1] - [剩余数量：54] - [当前时间：22:19:58]

[ok] - A7378
[尝试次数：1] - [剩余数量：53] - [当前时间：22:19:59]

[ok] - A7379
[尝试次数：1] - [剩余数量：52] - [当前时间：22:19:59]

[ok] - A7219
[尝试次数：1] - [剩余数量：51] - [当前时间：22:19:59]

[ok] - A7220
[尝试次数：1] - [剩余数量：50] - [当前时间：22:20:01]

[ok] - AM017
[尝试次数：1] - [剩余数量：49] - [当前时间